In [37]:
from __future__ import print_function, division
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import numpy as np
from DataPreProcessV import dataPreProcess

In [38]:
trainFilesName, validFilesName, testFilesName = [], [], []

for i in range(22):
    for j in range(3):
        if i < 9:
            fn = '../../P00%s/P00%s-00%s.xlsx' %(i+1, i+1, j+5)
        else:
            n = '../../P0%s/P0%s-00%s.xlsx' %(i+1, i+1, j+5)
        trainFilesName.append(fn)
    if i < 9:
        validFilesName.append('../../P00%s/P00%s-00%s.xlsx' %(i+1, i+1, j+6))
        testFilesName.append('../../P00%s/P00%s-00%s.xlsx' %(i+1, i+1, j+7))
    else:
        validFilesName.append('../../P0%s/P0%s-00%s.xlsx' %(i+1, i+1, j+6))
        testFilesName.append('../../P0%s/P0%s-00%s.xlsx' %(i+1, i+1, j+7))

print(trainFilesName)
print(validFilesName)
print(testFilesName)

['../../P001/P001-005.xlsx', '../../P001/P001-006.xlsx', '../../P001/P001-007.xlsx', '../../P002/P002-005.xlsx', '../../P002/P002-006.xlsx', '../../P002/P002-007.xlsx', '../../P003/P003-005.xlsx', '../../P003/P003-006.xlsx', '../../P003/P003-007.xlsx', '../../P004/P004-005.xlsx', '../../P004/P004-006.xlsx', '../../P004/P004-007.xlsx', '../../P005/P005-005.xlsx', '../../P005/P005-006.xlsx', '../../P005/P005-007.xlsx', '../../P006/P006-005.xlsx', '../../P006/P006-006.xlsx', '../../P006/P006-007.xlsx', '../../P007/P007-005.xlsx', '../../P007/P007-006.xlsx', '../../P007/P007-007.xlsx', '../../P008/P008-005.xlsx', '../../P008/P008-006.xlsx', '../../P008/P008-007.xlsx', '../../P009/P009-005.xlsx', '../../P009/P009-006.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-007.xlsx', '../../P009/P009-00

In [ ]:

ColumnsName = [['Right Elbow Ulnar Deviation/Radial Deviation', 'Right Elbow Pronation/Supination', 'Right Elbow Flexion/Extension',
                'Left Elbow Ulnar Deviation/Radial Deviation', 'Left Elbow Pronation/Supination', 'Left Elbow Flexion/Extension'],
             ['Right Hand x', 'Right Hand y','Right Hand z',
              'Left Hand x', 'Left Hand y', 'Left Hand z']]
sheetName = ['Joint Angles ZXY', 'Segment Position']


trainAllData, trainAllLabels = dataPreProcess(trainFilesName, ColumnsName, sheetName, leadtime=True, t=3)
valiAllData, valiAllLabels = dataPreProcess(validFilesName, ColumnsName, sheetName, leadtime=True, t=3)
testAllData, testAllLabels = dataPreProcess(testFilesName, ColumnsName, sheetName, leadtime=True, t=3)


trainAllData = trainAllData.reshape((len(trainAllData), 1 , 1, 48))
trainAllLabels = trainAllLabels.reshape((len(trainAllLabels),))
tensor_x = torch.Tensor(trainAllData) # transform to torch tensor
tensor_y = torch.Tensor(trainAllLabels).long()
train_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64) # create your dataloader

valiAllData = valiAllData.reshape((len(valiAllData), 1 , 1, 48))
valiAllLabels = valiAllLabels.reshape((len(valiAllLabels),))
tensor_x = torch.Tensor(valiAllData) # transform to torch tensor
tensor_y = torch.Tensor(valiAllLabels).long()
vali_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
vali_dataloader = DataLoader(vali_dataset, shuffle=True, batch_size=64) # create your dataloader

testAllData = testAllData.reshape((len(testAllData), 1 , 1, 48))
testAllLabels = testAllLabels.reshape((len(testAllLabels),))
tensor_x = torch.Tensor(testAllData) # transform to torch tensor
tensor_y = torch.Tensor(testAllLabels).long()
test_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=64) # create your dataloader

AllData, AllDataLen = {}, {}
AllData['train'] =train_dataloader
AllData['vali'] =vali_dataloader
AllData['test'] = test_dataloader
AllDataLen['train'] = len(trainAllData)
AllDataLen['vali'] = len(valiAllData)
AllDataLen['test'] = len(testAllData)

dataloaders = {x: AllData[x] for x in ['train', 'vali', 'test']}
dataset_sizes = {x: AllDataLen[x] for x in ['train', 'vali', 'test']}

In [ ]:
#References
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = models.googlenet(pretrained=True) #load googlenet.

num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 11) ###errors

# Change the ouptlayzersize
addNet1 = nn.ConvTranspose2d(1, 3, kernel_size=(224, 38), stride=(1, 4), padding=(0, 1))
addNet2 = nn.BatchNorm2d(3, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
addNets = nn.Sequential(addNet1, addNet2)
model_ft = nn.Sequential(addNets, model_ft)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft

In [ ]:
trainingRecords = {}
trainingRecords['Epoch'] = []
trainingRecords['Training loss'], trainingRecords['Training ACC'] = [], []
trainingRecords['Vali loss'], trainingRecords['Vali ACC'] = [], []

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs+1):
        epsince = time.time()
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'vali']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(inputs)
                #print(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #print(outputs)
                    #print(outputs.shape)
                    #print(labels.shape)
                    loss = criterion(outputs, labels)
    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            # Save the training records 
            trainingRecords[Epoch'].append(epoch+1)
            if phase == 'train':
                trainingRecords['Training loss'].append(epoch_loss)
                trainingRecords['Training ACC'].append(epoch_acc)
            else:
                trainingRecords['Vali loss'].append(epoch_loss)
                trainingRecords['Vali ACC'].append(epoch_acc)
                
            # deep copy the model
            if phase == 'vali' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        #Save model for each epoch
        PATH = './trainedParameters/GoogLeNetEP%s.pth' %(epoch+1)
        torch.save(copy.deepcopy(model.state_dict()), PATH)
        
        eptime_elapsed = time.time() - epsince
        print('Training epoch complete in {:.0f}m {:.0f}s'.format(
                eptime_elapsed // 60, eptime_elapsed % 60))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

ep = 100
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=ep)

In [ ]:
correct = 0
total = 0
nb_classes = 11
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in dataloaders['train']:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = model_ft(images)
        predicted = torch.round(outputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        
print('Accuracy of the network on the %s train images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)
b= torch.sum(confusion_matrix, dim=1)
#print(confusion_matrix.int()/torch.reshape(b, (-1, 1))*100) #Normalizing
cf = np.array(confusion_matrix/torch.reshape(b, (-1, 1))*100).astype(int)
print(cf)

In [ ]:
correct = 0
total = 0
nb_classes = 11
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in dataloaders['vali']:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = model_ft(images)
        predicted = torch.round(outputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        
print('Accuracy of the network on the %s vali images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)
b= torch.sum(confusion_matrix, dim=1)
#print(confusion_matrix/torch.reshape(b, (-1, 1))) #Normalizing
cf = np.array(confusion_matrix/torch.reshape(b, (-1, 1))*100).astype(int)
print(cf)

In [ ]:
correct = 0
total = 0
nb_classes = 11
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = model_ft(images)
        predicted = torch.round(outputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        
print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)
b= torch.sum(confusion_matrix, dim=1)
#print(confusion_matrix/torch.reshape(b, (-1, 1))) #Normalizing
cf = np.array(confusion_matrix/torch.reshape(b, (-1, 1))*100).astype(int)
print(cf)